In [41]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [43]:
os.chdir('C:/Users/doudi/OneDrive/Documents/ntcir15/Dataset/New_DialEval-1')
dev = pd.read_json(r'dev_cn.json', encoding='utf8')

In [44]:
def get_nugget(dataframe):   
    target = {'nugget'}
    dicts = []
    for item in tqdm(dataframe['annotations']):
        sub_dicts = []
        for element in item:
            sub_dicts.append({key:value for key,value in element.items() if key in target}['nugget'])
        dicts.append(sub_dicts)

    dataframe['nuggets'] = dicts # dis is for anno nugget list
    
    return dataframe

In [45]:
def shaping(dataframe):
    length = []
    for i in tqdm(dataframe['turns']):
        length.append(len(i))
    Id = dataframe['id'].tolist()

    Fin_Id = sum([[s] * n for s, n in zip(Id, length)], [])

    turns_list = dataframe['turns'].tolist()
    
    Fin_turns_anno = []
    for x,y in tqdm(zip(turns_list,dataframe['nuggets'])):
        for q in range(len(x)):
            Fin_turns_anno.append(list(x[q].values())+[i[q] for i in y])
    
    return Fin_Id, Fin_turns_anno

In [46]:
def stacking(Fin_Id, Fin_turns_anno):    
    train_clean = pd.DataFrame({'id': Fin_Id,'info': Fin_turns_anno})
    # train_clean.head()
    train_df = pd.DataFrame(train_clean['info'].values.tolist(), columns=['sender','utterance','n1','n2','n3','n4','n5','n6','n7','n8','n9','n10','n11','n12','n13','n14','n15','n16','n17','n18','n19'])
    train_df['id'] = train_clean['id']
    train_df = train_df[['id','sender','utterance','n1','n2','n3','n4','n5','n6','n7','n8','n9','n10','n11','n12','n13','n14','n15','n16','n17','n18','n19']]
    
    return train_df

In [47]:
two = get_nugget(dev)
Fin, Fin_turns = shaping(two)
df1 = stacking(Fin, Fin_turns)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [56]:
temp = []
for i in df1['id']:
    temp.append(str(i))
# temp[0:5]
df1['id'] = temp

In [57]:
df1.shape

(1755, 22)

In [60]:
import numpy as np

os.chdir('C:/Users/doudi/OneDrive/Documents/ntcir15/eval')
devset = pd.read_excel('200514_dev.xlsx', encoding='utf-8')

devset['id']=df1['id']

id       4227729258237823
CNUG            0.0526316
CNUG*                   0
CNUG0            0.157895
CNaN             0.789474
HNUG                    0
HNUG*                   0
HNaN                    0
Name: 0, dtype: object

In [64]:
def trans_json(dataset):
    data = []
    dict1 = {"nugget":[{"CNUG0":0, "CNUG":0, "CNUG*":0, "CNaN":0},
                       {"HNUG":0, "HNUG*":0, "HNaN":0}],"id":'0'}
    for v in range(len(dataset)):
        # 每一行資料形成一個列表
        values = dataset.loc[v]   
        if str(int(values[0])) == dict1["id"]:
            if values[1] == 0 and values[2] == 0 and values[3] == 0 and values[4] == 0:
                dict1["nugget"].append({"HNUG":values[5], "HNUG*":values[6], "HNaN":values[7]})
            else:
                dict1["nugget"].append({"CNUG0":values[3], "CNUG":values[1], "CNUG*":values[2], "CNaN":values[4]})
        else:
            data.append(dict1)
            if values[1] == 0 and values[2] == 0 and values[3] == 0 and values[4] == 0:
                dict1 = {"nugget":[{"HNUG":values[5], "HNUG*":values[6], "HNaN":values[7]}], 
                            "id":str(int(values[0]))}
            else:
                dict1 = {"nugget":[{"CNUG0":values[3], "CNUG":values[1], "CNUG*":values[2], "CNaN":values[4]}], 
                            "id":str(int(values[0]))}
    data.append(dict1)

        #把最前面的空dict1給刪掉
    del data[0]
    
    return data

In [65]:
dev_data = trans_json(devset)
dev_data

[{'nugget': [{'CNUG0': 0.15789473684210525,
    'CNUG': 0.05263157894736842,
    'CNUG*': 0.0,
    'CNaN': 0.7894736842105263},
   {'HNUG': 0.15789473684210525, 'HNUG*': 0.0, 'HNaN': 0.8421052631578947},
   {'CNUG0': 0.0,
    'CNUG': 0.15789473684210525,
    'CNUG*': 0.0,
    'CNaN': 0.8421052631578947},
   {'HNUG': 0.15789473684210525, 'HNUG*': 0.0, 'HNaN': 0.8421052631578947}],
  'id': '4227729258237823'},
 {'nugget': [{'CNUG0': 0.7894736842105263,
    'CNUG': 0.05263157894736842,
    'CNUG*': 0.0,
    'CNaN': 0.15789473684210525},
   {'HNUG': 0.5263157894736842,
    'HNUG*': 0.10526315789473684,
    'HNaN': 0.3684210526315789},
   {'CNUG0': 0.05263157894736842,
    'CNUG': 0.631578947368421,
    'CNUG*': 0.0,
    'CNaN': 0.3157894736842105},
   {'HNUG': 0.6842105263157895, 'HNUG*': 0.0, 'HNaN': 0.3157894736842105},
   {'CNUG0': 0.0,
    'CNUG': 0.6842105263157895,
    'CNUG*': 0.0,
    'CNaN': 0.3157894736842105},
   {'HNUG': 0.5789473684210527, 'HNUG*': 0.0, 'HNaN': 0.4210526315789

In [66]:
# write file
with open('dev_eval_2.json', 'w', encoding='utf-8') as f: 
    #ensure_ascii=False顯示中文,indent=2縮排為2
    f.write(json.dumps(dev_data, ensure_ascii=False, indent=2))